This notebook demonstrates the Policy Layer in action. It proves to stakeholders that you have controls in place.

In [1]:
import sys

sys.path.append("..")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.pricing_engine import LoanPricingEngine
import seaborn as sns

# 1. Initialize Engine
engine = LoanPricingEngine(
    risk_model_path="../models/risk_model_xgb.pkl",
    elasticity_model_path="../models/elasticity_model_logit.pkl",
)

print("Engine with Governance Layer Loaded")
print("Current Policy Config:", engine.policy_config)

Engine with Governance Layer Loaded
Current Policy Config: {'GLOBAL_MIN_RATE': 0.05, 'GLOBAL_MAX_RATE': 0.35, 'MAX_PD_THRESHOLD': 0.2, 'PRIME_MAX_RATE': 0.2, 'MIN_PROFIT_MARGIN': 50}


## 1. The "Prime Cap" Violation

Scenario: The ML model suggests a rate that is "fair" mathematically but too high for our brand reputation with VIP customers. 

Expected Result: The engine should intercept the rate and cap it at the configured limit (e.g., forcing it down from 10.3% to 9.8%).

In [2]:
print("\n--- TEST 1: PRIME CAP OVERRIDE ---")


prime_borrower = {
    "risk_score_norm": 0.95,
    "annual_inc": 150000,
    "dti": 0.10,
    "LoanOriginalAmount": 35000,
    "revol_util": 10,
    "term_years": 3,
    "emp_length": 5,
    "home_ownership_RENT": 0,
    "purpose_debt_consolidation": 1,
    "total_acc": 20,
    "inq_last_6mths": 0,
}


natural_result = engine.get_optimal_rate(prime_borrower)
natural_rate = natural_result["optimal_rate"]
print(f"Model naturally wants: {natural_rate:.2%}")


original_prime_cap = engine.policy_config["PRIME_MAX_RATE"]
test_cap = natural_rate - 0.005
engine.policy_config["PRIME_MAX_RATE"] = test_cap

print(
    f"Setting Artificial Cap:  {test_cap:.2%} (Verified Config: {engine.policy_config['PRIME_MAX_RATE']:.2%})"
)

decision = engine.get_optimal_rate(prime_borrower)

print(f"Final Governed Rate:    {decision['optimal_rate']:.2%}")
print(f"Policy Notes:           {decision['policy_notes']}")


if abs(decision["optimal_rate"] - test_cap) < 0.001:
    print("SUCCESS: Prime Cap override worked.")
else:
    print("FAIL: Rate was not capped correctly.")


engine.policy_config["PRIME_RATE_CAP"] = original_prime_cap


--- TEST 1: PRIME CAP OVERRIDE ---
Model naturally wants: 12.00%
Setting Artificial Cap:  11.50% (Verified Config: 11.50%)
Final Governed Rate:    11.50%
Policy Notes:           ['Capped at Prime Max (12%)']
SUCCESS: Prime Cap override worked.


## 2: The "Global Usury Limit" (Regulatory Cap)

Scenario: A risky borrower's mathematically optimal rate is > 20%, but the state regulator caps interest at 15%. 

Expected Result: The engine must clamp the rate to 15%.

In [3]:
print("\n--- TEST 2: GLOBAL USURY LIMIT ---")

risky_borrower = {
    "risk_score_norm": 0.55,
    "annual_inc": 80000,
    "dti": 0.40,
    "LoanOriginalAmount": 20000,
    "revol_util": 60,
    "term_years": 3,
    "emp_length": 5,
    "home_ownership_RENT": 1,
    "purpose_debt_consolidation": 1,
    "total_acc": 10,
    "inq_last_6mths": 1,
}


original_global_cap = engine.policy_config["GLOBAL_MAX_RATE"]
engine.policy_config["GLOBAL_MAX_RATE"] = 0.15

print(f"Setting Global Cap to 15.00%...")


decision = engine.get_optimal_rate(risky_borrower)

print(f"Final Governed Rate:    {decision['optimal_rate']:.2%}")
print(f"Expected Profit:        ${decision['max_profit']:.2f}")
print(f"Policy Notes:           {decision['policy_notes']}")


if decision["optimal_rate"] == 0.15:
    print("SUCCESS: Global Cap applied correctly.")
elif decision["decision"] == "REJECT_ECONOMICS":
    print("INCONCLUSIVE: Loan rejected for profitability. Increase Loan Amount.")
else:
    print(f"FAIL: Expected 15%, got {decision['optimal_rate']:.2%}")


engine.policy_config["GLOBAL_MAX_RATE"] = original_global_cap


--- TEST 2: GLOBAL USURY LIMIT ---
Setting Global Cap to 15.00%...
Final Governed Rate:    15.00%
Expected Profit:        $2423.20
Policy Notes:           []
SUCCESS: Global Cap applied correctly.


## 3: The "Profitability Floor" (Rejection)

Scenario: The policy forces a rate so low (e.g., 4%) that the loan is no longer profitable (Cost of Funds is 4%). 

Expected Result: The engine should change the decision from APPROVE to REJECT_ECONOMICS.

In [4]:
print("\n--- TEST 3: UNPROFITABLE REJECTION ---")

engine.policy_config["GLOBAL_MAX_RATE"] = 0.04

print(f"Setting Global Cap to 4.00% (Below Break-Even)...")

decision = engine.get_optimal_rate(risky_borrower)

print(f"Decision:               {decision['decision']}")
print(f"Policy Notes:           {decision['policy_notes']}")
print(f"Expected Profit:        ${decision['max_profit']:.2f}")


if decision["decision"] == "REJECT_ECONOMICS":
    print("SUCCESS: Unprofitable loan rejected.")
else:
    print(f"FAIL: Loan was not rejected (Status: {decision['decision']})")


engine.policy_config["GLOBAL_MAX_RATE"] = 0.36
print("All configs reset.")


--- TEST 3: UNPROFITABLE REJECTION ---
Setting Global Cap to 4.00% (Below Break-Even)...
Decision:               REJECT_ECONOMICS
Policy Notes:           ['Expected profit below minimum margin']
Expected Profit:        $-1705.99
SUCCESS: Unprofitable loan rejected.
All configs reset.
